# **Project Sistem Rekomendasi : Metode Hibrida**

## Tujuan dan Manfaat Proyek

**Tujuan utama** dari proyek ini adalah untuk membangun sebuah sistem rekomendasi film yang dapat meningkatkan keterlibatan (engagement) pengguna dan memperpanjang durasi mereka di platform. Kami ingin menyediakan rekomendasi yang sangat personal dan akurat, yang terasa seperti kurasi dari seorang ahli film.

**Manfaat langsung** dari pencapaian tujuan ini adalah:

* Peningkatan Retensi Pengguna: Pengguna yang merasa platform memahami selera mereka cenderung akan kembali lagi dan tidak beralih ke layanan lain.

* Optimalisasi Konten: Wawasan dari model dapat membantu tim konten dalam mengambil keputusan yang didukung data tentang film apa yang harus ditambahkan ke katalog.

* Peluang Monetisasi: Durasi tontonan yang lebih lama dan tingkat kunjungan yang lebih sering membuka peluang untuk pendapatan dari iklan atau peningkatan langganan.



#Import Data

Mengimpor semua pustaka (libraries) Python yang dibutuhkan untuk proyek.

`pandas`: Untuk memanipulasi dan menganalisis data dalam format tabel (DataFrame).

`sklearn.feature_extraction.text.TfidfVectorizer`: Untuk mengubah data teks (genre film) menjadi vektor numerik menggunakan metode TF-IDF. Ini adalah inti dari model content-based.

`sklearn.metrics.pairwise.cosine_similarity`: Untuk menghitung kemiripan antar film berdasarkan vektor genre mereka.

`fastai.collab & fastai.tabular.all`: Pustaka tingkat tinggi yang mempermudah pembuatan model collaborative filtering dengan cepat.

In [56]:
# Import pustaka yang diperlukan
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Import pustaka yang diperlukan
import pandas as pd
from fastai.collab import *
from fastai.tabular.all import *
from pathlib import Path

Perintah ini menggunakan Git untuk mengkloning (mengunduh) sebuah repositori dari GitHub. Repositori ini berisi dataset yang akan digunakan (`movies.csv, ratings.csv`, dll.). File-file tersebut akan disalin ke lingkungan kerja notebook.

In [57]:
#Clone repository dari github untuk mengakses dataset yang telah diunggah di github sebelumnya
!git clone https://github.com/Andishafira/Dicoding.git

fatal: destination path 'Dicoding' already exists and is not an empty directory.


Kode ini mendefinisikan lokasi (path) dari empat file CSV yang baru saja diunduh dan memuatnya ke dalam DataFrame pandas.

`df_movie`: Berisi informasi film (movieId, title, genres).

`df_ratings`: Berisi data rating yang diberikan pengguna ke film (userId, movieId, rating).

`df_tags dan df_links`: Data tambahan yang tidak digunakan dalam model akhir di notebook ini.

In [58]:
#Inisialisasi path dari masing masing csv

df_movie_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/movies.csv"
df_ratings_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/ratings.csv"
#df_tags_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/tags.csv"
#df_links_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/links.csv"

df_movie = pd.read_csv(df_movie_path)
df_ratings = pd.read_csv(df_ratings_path)
#df_tags = pd.read_csv(df_tags_path)
#df_links = pd.read_csv(df_links_path)

Kode ini digunakan untuk melakukan analisis data eksplorasi dasar untuk memahami isi dataset.

`df_movie.head() & df_ratings.head()`: Menampilkan 5 baris pertama dari tabel film dan rating untuk melihat struktur datanya.

`df_movie.info() & df_ratings.info()`: Memberikan ringkasan teknis tentang DataFrame, seperti jumlah data, nama kolom, dan tipe datanya. Ini berguna untuk memeriksa apakah ada data yang hilang (missing values).

`len(df_movie['genres'].unique())`: Menghitung jumlah kombinasi genre yang unik.

`all_genres.value_counts().head(10)`: Memisahkan genre yang digabung dengan | (misalnya, "Adventure|Animation|Children") menjadi genre individual, lalu menghitung dan menampilkan 10 genre paling umum di seluruh dataset (Drama, Comedy, Thriller, dll.).

In [59]:
# Menampilkan 5 baris pertama dari movies_df
print("--- 5 baris pertama dari movies.csv ---")
print(df_movie.head())

# Menampilkan informasi kolom dan tipe data dari movies_df
print("\n--- Informasi dataframe movies_df ---")
print(df_movie.info())

# Menampilkan 5 baris pertama dari ratings_df
print("\n--- 5 baris pertama dari ratings.csv ---")
print(df_ratings.head())

# Menampilkan informasi kolom dan tipe data dari ratings_df
print("\n--- Informasi dataframe ratings_df ---")
print(df_ratings.info())

# Memeriksa jumlah nilai unik pada kolom 'genres'
print("\n--- Jumlah genre unik ---")
print(len(df_movie['genres'].unique()))

# Mengambil 10 genre teratas
print("\n--- 10 genre teratas ---")
# Memisahkan genre terlebih dahulu untuk mendapatkan gambaran yang lebih akurat
all_genres = df_movie['genres'].str.split('|', expand=True).stack().reset_index(level=1, drop=True)
print(all_genres.value_counts().head(10))

--- 5 baris pertama dari movies.csv ---
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

--- Informasi dataframe movies_df ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory 

#Preprocessing Data

In [60]:
# Gabungkan data ratings dan movies
ratings = df_ratings.merge(df_movie)

Tabel `df_ratings` dan `df_movie` digabungkan menjadi satu DataFrame baru bernama `ratings`. Penggabungan ini dilakukan berdasarkan kolom `movieId` yang ada di kedua tabel. Hasilnya adalah sebuah tabel yang berisi `userId, rating, title,` dan `genres` dalam satu baris, yang memudahkan proses pelatihan model.

In [61]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [62]:
#Membagi Data
#split ratings to train df and test df (80:20)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

Dataset ratings dibagi menjadi dua bagian:

* **Data Latih (train_df)**: 80% dari data, digunakan untuk melatih model.

* **Data Uji (test_df)**: 20% dari data, digunakan untuk mengevaluasi seberapa baik performa model pada data yang belum pernah dilihat sebelumnya.

In [63]:
#TfidfVectorizer
# Dilakukan untuk mengubah data teks (genre film) menjadi representasi angka (vektor) yang bermakna
# Use the existing 'genres' column from df_movie directly
genres_data = df_movie['genres']

# Inisialisasi TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# Use the corrected genres_data for fitting and transforming
tfidf_matrix = tfidf_vectorizer.fit_transform(genres_data)

Hasilnya adalah sebuah matriks angka (tfidf_matrix) di mana setiap baris mewakili satu film dan setiap kolom mewakili satu genre unik, dengan nilai di dalamnya adalah skor bobot TF-IDF.

#Train Model

Sel ini adalah inti dari pemodelan, di mana dua jenis sistem rekomendasi dibangun.

1. **Model Collaborative Filtering** (dengan fastai):

`CollabDataLoaders.from_df`: Mempersiapkan data latih ke dalam format yang dibutuhkan oleh fastai.

`collab_learner`: Membuat model collaborative filtering. Model ini belajar pola dari interaksi pengguna-film secara keseluruhan.

`learn.fit_one_cycle(5, 5e-3)`: Memulai proses pelatihan model selama 5 epoch (siklus). Output tabel menunjukkan bahwa loss (kesalahan) pada data validasi semakin menurun, yang menandakan model sedang belajar.

2. **Model Content-Based Filtering** (dengan scikit-learn):

`TfidfVectorizer`: Mengonversi kolom genres dari setiap film menjadi representasi angka (vektor). Film dengan genre serupa akan memiliki vektor yang mirip (Tahapan ini ada di data preparation)

`cosine_similarity`: Menghitung skor kemiripan (antara -1 dan 1) untuk setiap pasang film berdasarkan vektor genre mereka. Hasilnya adalah matriks cosine_sim yang menyimpan skor ini.

In [64]:
# --- Model Collaborative Filtering (fastai) ---
# Persiapan data loaders
dls = CollabDataLoaders.from_df(
    train_df,
    user_name='userId',
    item_name='title',
    rating_name='rating',
    bs=64
)

# Latih model
learn = collab_learner(dls, n_factors=50, y_range=(0.5, 5.5), metrics=rmse)
#learn.lr_find()
learn.fit_one_cycle(5, 0.001)

epoch,train_loss,valid_loss,_rmse,time
0,1.141062,1.123719,1.060056,00:06
1,0.730116,0.814338,0.902407,00:06
2,0.650688,0.768635,0.876718,00:06
3,0.572365,0.756747,0.869912,00:06
4,0.582169,0.754741,0.868759,00:06


In [65]:
# --- Model Content-based (berbasis genre) ---

# Hitung Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

##Membuat Model Hybrid

Sel ini mendefinisikan fungsi `get_hybrid_recommendations` yang menggabungkan kekuatan kedua model sebelumnya.

1. Mendapatkan Film yang Belum Ditonton: Fungsi ini mengambil `user_id` dan mencari semua film yang belum pernah ia tonton.

2. Prediksi Rating (Collaborative): Model `fastai` digunakan untuk memprediksi rating yang mungkin akan diberikan pengguna pada film-film yang belum ia tonton.

3. Mencari Film Favorit (Benih Content-Based): Fungsi ini mencari film dengan rating tertinggi yang pernah diberikan oleh pengguna. Film ini akan menjadi "benih" untuk mencari film lain yang genrenya mirip.

4. Skor Kemiripan Genre (Content-Based): Berdasarkan film "benih" tadi, fungsi ini mengambil skor kemiripan genre dari matriks `cosine_sim` untuk semua film lainnya.

5. Menghitung Skor Hibrida: Skor akhir dihitung dengan menggabungkan prediksi rating (dari model collaborative) dan skor kemiripan genre (dari model content-based) menggunakan bobot tertentu (`collaborative_weight dan content_weight`).

6. Mengembalikan Top-N Rekomendasi: Fungsi mengembalikan daftar film teratas berdasarkan skor hibrida tertinggi.

In [66]:
def get_hybrid_recommendations(user_id, num_recommendations=10, collaborative_weight=0.6, content_weight=0.4):
    """
    Memberikan rekomendasi film hibrida untuk pengguna tertentu.

    Fungsi ini menggabungkan:
    1. Prediksi rating dari model Collaborative Filtering.
    2. Kemiripan genre dari model Content-based Filtering.

    Args:
        user_id (int): ID pengguna.
        num_recommendations (int): Jumlah rekomendasi yang diinginkan.
        collaborative_weight (float): Bobot untuk skor collaborative (prediksi rating).
        content_weight (float): Bobot untuk skor content-based (kemiripan genre).

    Returns:
        pd.DataFrame: Daftar film yang direkomendasikan dengan skor hibrida.
    """
    # 1. Mendapatkan daftar film yang belum pernah ditonton pengguna
    user_rated_movies = train_df[train_df['userId'] == user_id]['title'].tolist()
    all_movies_titles = df_movie['title'].tolist()
    unrated_movies = [title for title in all_movies_titles if title not in user_rated_movies]

    # Pilih film acak dari yang belum ditonton untuk diprediksi (untuk efisiensi)
    unrated_movies_sample = pd.Series(unrated_movies).tolist()

    # 2. Menggunakan model Collaborative untuk memprediksi rating
    user_tensor = torch.tensor([dls.classes['userId'].o2i[user_id]] * len(unrated_movies_sample))
    movie_titles_tensor = torch.tensor([dls.classes['title'].o2i[title] for title in unrated_movies_sample])

    # Concatenate user and movie tensors
    collab_input = torch.stack([user_tensor, movie_titles_tensor], dim=1)

    with torch.no_grad():
        collab_preds = learn.model.forward(collab_input)
    collab_preds = collab_preds.numpy()

    collab_df = pd.DataFrame({'title': unrated_movies_sample, 'predicted_rating': collab_preds.flatten()}) # Flatten to match the shape

    # 3. Mendapatkan film favorit pengguna (sebagai "benih" untuk model Content-based)
    # Kita ambil 5 film dengan rating tertinggi dari pengguna
    user_top_movies = train_df[(train_df['userId'] == user_id) & (train_df['rating'] >= 4)].sort_values(by='rating', ascending=False).head(5)

    if user_top_movies.empty:
        print("Pengguna ini belum memiliki rating film tinggi. Menggunakan rekomendasi Collaborative saja.")
        # Add content similarity scores of 0 for all recommendations if no top movies
        collab_df['content_sim_score'] = 0.0
        collab_df['hybrid_score'] = collaborative_weight * collab_df['predicted_rating']
        return collab_df.sort_values(by='hybrid_score', ascending=False).head(num_recommendations)


    seed_movie_title = user_top_movies['title'].iloc[0]

    # 4. Menghitung skor kemiripan konten untuk re-ranking
    # Dapatkan indeks film 'seed'
    seed_movie_idx = df_movie[df_movie['title'] == seed_movie_title].index[0]

    # Dapatkan skor kemiripan dari film seed dengan semua film lain
    content_sim_scores = cosine_sim[seed_movie_idx]

    # Gabungkan dengan dataframe prediksi
    hybrid_recs = collab_df.merge(df_movie[['title']], on='title')
    hybrid_recs['content_sim_score'] = hybrid_recs['title'].apply(
        lambda x: content_sim_scores[df_movie[df_movie['title'] == x].index[0]]
    )

    # 5. Menggabungkan skor untuk mendapatkan skor hibrida
    hybrid_recs['hybrid_score'] = (
        collaborative_weight * hybrid_recs['predicted_rating'] +
        content_weight * (hybrid_recs['content_sim_score'] * 5) # Kalikan dengan 5 agar skalanya mirip
    )

    # Urutkan berdasarkan skor hibrida dan tampilkan hasilnya
    final_recs = hybrid_recs.sort_values(by='hybrid_score', ascending=False).head(num_recommendations)
    return final_recs[['title', 'hybrid_score', 'predicted_rating', 'content_sim_score']]

##Contoh Testing

Kode ini menunjukkan cara menggunakan fungsi hibrida. Fungsi `get_hybrid_recommendations` dipanggil untuk pengguna dengan ID 197. Hasilnya adalah 10 film yang direkomendasikan, diurutkan berdasarkan `hybrid_score` tertinggi.

In [67]:
# Contoh penggunaan:
user_id_to_recommend = 197
print(f"--- Rekomendasi Hibrida untuk Pengguna {user_id_to_recommend} ---")
recommendations = get_hybrid_recommendations(user_id_to_recommend)
print(recommendations)

--- Rekomendasi Hibrida untuk Pengguna 197 ---
                                                                 title  \
6515                                                   Superbad (2007)   
378                                          Dazed and Confused (1993)   
1060                                            Raising Arizona (1987)   
2068  Monty Python's And Now for Something Completely Different (1971)   
974                                          This Is Spinal Tap (1984)   
2170                                   Ferris Bueller's Day Off (1986)   
4556                         Monty Python's The Meaning of Life (1983)   
2962                               Planes, Trains & Automobiles (1987)   
3887                                                Top Secret! (1984)   
189                                                      Clerks (1994)   

      hybrid_score  predicted_rating  content_sim_score  
6515      4.261823          3.769704                1.0  
378       4.230481    

#Evaluasi Model

Untuk mengukur kinerja model secara objektif, fungsi `evaluate_hybrid_recommender` dibuat.

1. **Metrik** : Fungsi ini menggunakan **Precision@k** (berapa persen dari k rekomendasi yang relevan) dan **Recall@k**(berapa persen dari total film relevan yang berhasil direkomendasikan).

2. **Proses** : Fungsi ini berjalan pada data uji (`test_df`). Untuk setiap pengguna sampel:

    * Menentukan "film relevan" (film yang diberi rating tinggi, misalnya >= 4.0).

    * Menghasilkan 10 rekomendasi teratas menggunakan fungsi hibrida.

    * Membandingkan film yang direkomendasikan dengan film yang relevan untuk menghitung "hits".

    * Menghitung Precision dan Recall.

3. **Hasil** : Sel terakhir menjalankan evaluasi dan mencetak Rata-rata Precision@10: 0.0380 dan Rata-rata Recall@10: 0.0285.

In [68]:
def evaluate_hybrid_recommender(model, ratings_df, test_df, k=10, min_rating=4.0):
    """
    Mengevaluasi sistem rekomendasi hibrida menggunakan metrik Precision@k dan Recall@k.

    Args:
        model: Model fastai yang sudah dilatih.
        ratings_df (pd.DataFrame): Dataframe rating lengkap.
        k (int): Jumlah item teratas yang akan dipertimbangkan untuk rekomendasi.
        min_rating (float): Rating minimum untuk dianggap "relevan".

    Returns:
        tuple: Precision@k dan Recall@k rata-rata.
    """
    all_users = test_df['userId'].unique()
    all_movies = ratings_df['title'].unique()

    precision_scores = []
    recall_scores = []

    # Ambil sampel beberapa pengguna untuk evaluasi yang efisien
    sample_users = pd.Series(all_users).sample(n=min(50, len(all_users)), random_state=42)

    for user_id in sample_users:
        # Dapatkan film yang sudah dirating pengguna
        user_ratings = test_df[test_df['userId'] == user_id]

        # Pisahkan film relevan (rating tinggi) dari film yang tidak relevan
        relevant_movies = user_ratings[user_ratings['rating'] >= min_rating]['title'].tolist()
        print(f"Pengguna {user_id} memiliki {len(relevant_movies)} film relevan.")
        # Dapatkan rekomendasi hibrida
        # Karena ini simulasi, kita gunakan fungsi rekomendasi sebelumnya
        recommendations = get_hybrid_recommendations(user_id, num_recommendations=k)
        print(f"Rekomendasi untuk pengguna {user_id}: {recommendations['title'].tolist()}")
        if recommendations is None:
            continue

        recommended_movies = recommendations['title'].tolist()

        # Hitung irisan antara film yang direkomendasikan dan film yang relevan
        hits = len(set(recommended_movies) & set(relevant_movies))
        print(f"Pengguna {user_id} memiliki {hits} hits.")
        # Hitung Precision@k
        precision = hits / len(recommended_movies) if len(recommended_movies) > 0 else 0
        precision_scores.append(precision)

        # Hitung Recall@k
        recall = hits / len(relevant_movies) if len(relevant_movies) > 0 else 0
        recall_scores.append(recall)

    # Hitung rata-rata Precision dan Recall
    avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0

    return avg_precision, avg_recall



In [69]:
# Uji coba dengan k=10 dan rating relevan >= 4
avg_prec, avg_rec = evaluate_hybrid_recommender(learn, ratings, test_df, k=10, min_rating=4.0)
print(f"Rata-rata Precision@{10}: {avg_prec:.4f}")
print(f"Rata-rata Recall@{10}: {avg_rec:.4f}")

Pengguna 428 memiliki 4 film relevan.
Rekomendasi untuk pengguna 428: ['Graduate, The (1967)', 'Adaptation (2002)', 'Harold and Maude (1971)', 'Philadelphia Story, The (1940)', 'Lost in Translation (2003)', 'Moonrise Kingdom (2012)', 'Manhattan (1979)', 'Hannah and Her Sisters (1986)', 'Apartment, The (1960)', 'Sideways (2004)']
Pengguna 428 memiliki 0 hits.
Pengguna 582 memiliki 9 film relevan.
Rekomendasi untuk pengguna 582: ['Watchmen (2009)', 'V for Vendetta (2006)', 'Edge of Tomorrow (2014)', 'Minority Report (2002)', 'Super 8 (2011)', 'Donnie Darko (2001)', 'Clockwork Orange, A (1971)', 'Matrix, The (1999)', 'Spider-Man 2 (2004)', 'Batman Begins (2005)']
Pengguna 582 memiliki 1 hits.
Pengguna 215 memiliki 14 film relevan.
Rekomendasi untuk pengguna 215: ['Godfather, The (1972)', 'Godfather: Part II, The (1974)', 'American History X (1998)', 'Goodfellas (1990)', 'Three Billboards Outside Ebbing, Missouri (2017)', 'Casino (1995)', 'On the Waterfront (1954)', 'Gran Torino (2008)', '

**Dari data yang disajikan, terlihat bahwa kinerja sistem rekomendasi ini, yang diukur dengan metrik Precision@k dan Recall@k pada k=10, masih sangat rendah.**

* Rata-rata Precision@10 sebesar 0.0380 menunjukkan bahwa, rata-rata, hanya sekitar 3.8% dari film yang direkomendasikan kepada pengguna adalah film yang relevan (film yang mereka sukai). Dengan kata lain, dari 10 film yang direkomendasikan, hanya sekitar 0.38 film (kurang dari satu) yang benar-benar relevan.

* Rata-rata Recall@10 sebesar 0.0285 menunjukkan bahwa, rata-rata, sistem hanya berhasil menemukan sekitar 2.85% dari total film relevan yang disukai pengguna. Ini berarti sistem gagal menemukan sebagian besar film relevan yang ada dalam data pengguna.

Meskipun model ini berhasil memberikan rekomendasi yang terlihat masuk akal secara tematis (misalnya, rekomendasi film-film aksi/sci-fi untuk Pengguna 582 dan 599, atau film kriminal untuk Pengguna 215), jumlah "hits" (film yang direkomendasikan dan juga relevan) sangat sedikit. Dari 32 pengguna yang dievaluasi, hanya 9 pengguna yang mendapatkan setidaknya satu film relevan di antara 10 rekomendasi teratas mereka.

**Analisis Lebih Lanjut**

Hasil ini menunjukkan bahwa ada beberapa area yang perlu diperbaiki:

* Keterbatasan Data: Jumlah pengguna dan film yang digunakan untuk evaluasi ini mungkin tidak cukup besar untuk memberikan gambaran yang akurat. Selain itu, sebaran data rating mungkin tidak merata, menyebabkan beberapa pengguna memiliki sedikit atau bahkan nol film relevan (seperti Pengguna 207 dan 496).

* Masalah Cold-Start: Sebagian besar pengguna dalam evaluasi ini (seperti Pengguna 428, 215, 461, 197, dst.) memiliki hits nol. Ini bisa menjadi indikasi masalah cold-start, di mana model tidak memiliki cukup data riwayat rating untuk pengguna tersebut, sehingga rekomendasinya tidak akurat.

* Bobot Model Hibrida: Bobot yang diberikan pada model Collaborative dan Content-based (jika ini adalah model hibrida) mungkin tidak optimal. Model mungkin terlalu condong ke salah satu sisi, sehingga gagal memanfaatkan kekuatan gabungan dari kedua pendekatan.

* Definisi "Relevan": Mungkin ada masalah dalam definisi "film relevan". Jika hanya film dengan rating 4 atau 5 yang dianggap relevan, model mungkin kesulitan jika sebagian besar rating pengguna berada di angka 3.

Secara keseluruhan, meskipun proyek ini berhasil membangun sebuah alur kerja rekomendasi, hasil evaluasi menunjukkan bahwa model ini belum siap untuk digunakan di lingkungan produksi. Diperlukan iterasi lebih lanjut pada model, data, dan strategi evaluasi untuk meningkatkan performa secara signifikan.

##Perbandingan Metode Hibryd dengan CF murni dan CBF murni

###Collaborative Filtering Murni

In [70]:
def get_collaborative_recommendations(user_id, num_recommendations=10):
    """
    Memberikan rekomendasi murni Collaborative Filtering (berdasarkan predicted_rating tertinggi).
    """
    # 1. Mendapatkan daftar film yang belum pernah ditonton pengguna (dari data latih)
    user_rated_movies = train_df[train_df['userId'] == user_id]['title'].tolist()
    all_movies_titles = df_movie['title'].tolist()
    unrated_movies = [title for title in all_movies_titles if title not in user_rated_movies]
    unrated_movies_sample = pd.Series(unrated_movies).tolist()

    # 2. Menggunakan model Collaborative (learn) untuk memprediksi rating
    user_tensor = torch.tensor([dls.classes['userId'].o2i[user_id]] * len(unrated_movies_sample))
    movie_titles_tensor = torch.tensor([dls.classes['title'].o2i[title] for title in unrated_movies_sample])
    collab_input = torch.stack([user_tensor, movie_titles_tensor], dim=1)

    with torch.no_grad():
        collab_preds = learn.model.forward(collab_input)
    collab_preds = collab_preds.numpy()

    collab_df = pd.DataFrame({'title': unrated_movies_sample, 'predicted_rating': collab_preds.flatten()})

    # 3. Urutkan berdasarkan predicted_rating saja (bukan hybrid_score)
    final_recs = collab_df.sort_values(by='predicted_rating', ascending=False).head(num_recommendations)

    return final_recs

def evaluate_cf_recommender(model, ratings_df, test_df, k=10, min_rating=4.0):
    """
    Mengevaluasi sistem rekomendasi CF murni menggunakan Precision@k dan Recall@k.
    """
    all_users = test_df['userId'].unique()
    precision_scores = []
    recall_scores = []

    sample_users = pd.Series(all_users).sample(n=min(50, len(all_users)), random_state=42)

    for user_id in sample_users:
        user_ratings = test_df[test_df['userId'] == user_id]
        relevant_movies = user_ratings[user_ratings['rating'] >= min_rating]['title'].tolist()

        if not relevant_movies:
            continue

        # Memanggil fungsi rekomendasi CF murni
        recommendations = get_collaborative_recommendations(user_id, num_recommendations=k)
        # -------------------------

        if recommendations is None or recommendations.empty:
            precision_scores.append(0)
            recall_scores.append(0)
            continue

        recommended_movies = recommendations['title'].tolist()
        hits = len(set(recommended_movies) & set(relevant_movies))

        precision = hits / len(recommended_movies) if len(recommended_movies) > 0 else 0
        precision_scores.append(precision)

        recall = hits / len(relevant_movies) if len(relevant_movies) > 0 else 0
        recall_scores.append(recall)

    avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0

    return avg_precision, avg_recall

In [71]:
# Evaluasi model CF Murni
print("--- Mengevaluasi Model Collaborative Filtering Murni ---")
avg_prec_cf, avg_rec_cf = evaluate_cf_recommender(learn, ratings, test_df, k=10, min_rating=4.0)
print(f"\nRata-rata Precision@10 (CF Murni): {avg_prec_cf:.4f}")
print(f"Rata-rata Recall@10 (CF Murni): {avg_rec_cf:.4f}")

--- Mengevaluasi Model Collaborative Filtering Murni ---

Rata-rata Precision@10 (CF Murni): 0.1000
Rata-rata Recall@10 (CF Murni): 0.0865


### Content-Based Filtering

In [72]:
def get_content_based_recommendations(user_id, num_recommendations=10):
    """
    Memberikan rekomendasi murni Content-Based Filtering (berdasarkan kemiripan genre).
    """
    # 1. Mendapatkan film favorit pengguna (benih) dari data latih
    user_top_movies = train_df[(train_df['userId'] == user_id) & (train_df['rating'] >= 4.0)].sort_values(by='rating', ascending=False).head(5)

    if user_top_movies.empty:
        # Jika pengguna tidak punya rating tinggi, kita tidak bisa memberi rekomendasi berbasis konten
        return None

    # Kita ambil 1 film teratas sebagai benih utama
    seed_movie_title = user_top_movies['title'].iloc[0]
    seed_movie_idx = df_movie[df_movie['title'] == seed_movie_title].index[0]

    # 2. Dapatkan skor kemiripan dari film benih ke SEMUA film lain
    content_sim_scores = cosine_sim[seed_movie_idx]

    # Ubah menjadi series pandas untuk memudahkan manipulasi
    all_movies_sim = pd.Series(content_sim_scores, index=df_movie['title'])

    # 3. Hapus film yang sudah ditonton pengguna
    user_rated_movies = train_df[train_df['userId'] == user_id]['title'].tolist()
    all_movies_sim = all_movies_sim.drop(user_rated_movies, errors='ignore')

    # 4. Urutkan berdasarkan skor kemiripan tertinggi
    final_recs = all_movies_sim.sort_values(ascending=False).head(num_recommendations)

    # Format output agar sama seperti fungsi lain (DataFrame)
    return pd.DataFrame({'title': final_recs.index, 'content_sim_score': final_recs.values})

def evaluate_cbf_recommender(ratings_df, test_df, k=10, min_rating=4.0):
    """
    Mengevaluasi sistem rekomendasi CBF murni menggunakan Precision@k dan Recall@k.
    """
    all_users = test_df['userId'].unique()
    precision_scores = []
    recall_scores = []

    sample_users = pd.Series(all_users).sample(n=min(50, len(all_users)), random_state=42)

    for user_id in sample_users:
        user_ratings = test_df[test_df['userId'] == user_id]
        relevant_movies = user_ratings[user_ratings['rating'] >= min_rating]['title'].tolist()

        if not relevant_movies:
            continue

        # Memanggil fungsi rekomendasi CBF murni
        recommendations = get_content_based_recommendations(user_id, num_recommendations=k)
        # -------------------------

        if recommendations is None or recommendations.empty:
            precision_scores.append(0)
            recall_scores.append(0)
            continue

        recommended_movies = recommendations['title'].tolist()
        hits = len(set(recommended_movies) & set(relevant_movies))

        precision = hits / len(recommended_movies) if len(recommended_movies) > 0 else 0
        precision_scores.append(precision)

        recall = hits / len(relevant_movies) if len(relevant_movies) > 0 else 0
        recall_scores.append(recall)

    avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0

    return avg_precision, avg_recall

In [73]:
# Evaluasi model CBF Murni
print("--- Mengevaluasi Model Content-Based Filtering Murni ---")
avg_prec_cbf, avg_rec_cbf = evaluate_cbf_recommender(ratings, test_df, k=10, min_rating=4.0)
print(f"\nRata-rata Precision@10 (CBF Murni): {avg_prec_cbf:.4f}")
print(f"Rata-rata Recall@10 (CBF Murni): {avg_rec_cbf:.4f}")

--- Mengevaluasi Model Content-Based Filtering Murni ---

Rata-rata Precision@10 (CBF Murni): 0.0063
Rata-rata Recall@10 (CBF Murni): 0.0028


### Hasil Analisis

In [74]:
# --- Pembuatan Tabel Perbandingan Hasil Evaluasi ---

# 1. Kumpulkan semua data metrik ke dalam dictionary
evaluation_summary_data = {
    'Model': [
        '1. Collaborative Filtering (Murni)',
        '2. Content-Based Filtering (Murni)',
        '3. Hybrid Re-ranking'
    ],
    'Precision@10': [avg_prec_cf, avg_prec_cbf, avg_prec],
    'Recall@10': [avg_rec_cf, avg_rec_cbf, avg_rec]
}

# 2. Buat DataFrame dari dictionary
df_evaluation_comparison = pd.DataFrame(evaluation_summary_data)

# 3. Atur kolom 'Model' sebagai index agar tabel lebih mudah dibaca
df_evaluation_comparison = df_evaluation_comparison.set_index('Model')

# 4. Tampilkan tabel hasil perbandingan
print("--- Tabel Perbandingan Metrik Evaluasi Model ---")
print(df_evaluation_comparison)

--- Tabel Perbandingan Metrik Evaluasi Model ---
                                    Precision@10  Recall@10
Model                                                      
1. Collaborative Filtering (Murni)       0.10000   0.086472
2. Content-Based Filtering (Murni)       0.00625   0.002760
3. Hybrid Re-ranking                     0.04800   0.039495


Berdasarkan hasil tersebut dapat dilihat jika metode collaborative filtering mendapatkan hasil yang paling baik dengan precision@10 sebesar 0.10000 dan recall@10 sebesar 0.086472. Angka ini menunjukkan bahwa dengan hanya melihat pola perilaku pengguna (apa yang disukai pengguna serupa), model ini berhasil menempatkan 1 dari 10 film yang benar-benar relevan di daftar rekomendasi. Ini membuktikan bahwa sinyal "kearifan kolektif" (apa yang disukai orang lain) adalah prediktor yang jauh lebih kuat daripada fitur genre. Model CF berhasil menemukan koneksi lintas-genre yang tidak bisa dilihat oleh CBF.

Sedangkan, Performa model content based filtering hampir nol. Presisi 0.6% pada dasarnya berarti model ini hampir tidak pernah memberikan rekomendasi yang benar dan performanya tidak jauh lebih baik dari tebakan acak. Ini membuktikan bahwa strategi CBF yang digunakan (mencari film yang mirip genrenya hanya dengan satu film favorit pengguna) adalah strategi yang sangat tidak efektif. Selera pengguna jauh lebih kompleks daripada hanya menyukai satu genre yang sama berulang kali.

# **Implementasi Sistem Rekomendasi**

Proyek ini berhasil membangun sistem rekomendasi film menggunakan metode hibrida. Meski demikian, berdasarkan analisis evaluasi, dapat disimpulkan bahwa model hibrida menunjukkan kinerja sub-optimal, dengan skor metrik performa yang lebih rendah dibandingkan model Collaborative Filtering (CF) murni.

Degradasi performa ini disebabkan oleh penggabungan sinyal dari model Content-Based Filtering (CBF) yang terbukti memiliki akurasi prediktif yang sangat rendah. Alokasi bobot yang signifikan (40%) pada komponen CBF yang tidak akurat ini secara efektif mendistorsi dan menurunkan kualitas peringkat relevan yang sebelumnya telah dihasilkan oleh model CF yang jauh lebih akurat. Dengan kata lain, intervensi dari prediktor yang lemah telah memberikan dampak negatif pada hasil akhir sistem.

## Strategi Bisnis

Sistem hibrida ini memiliki potensi besar untuk diintegrasikan ke dalam operasi bisnis. Berikut adalah beberapa strategi utama:

* Personalisasi Halaman Utama: Rekomendasi paling relevan akan ditampilkan di halaman utama, memastikan pengguna langsung disambut dengan konten yang mereka sukai. Ini dapat meningkatkan rata-rata sesi pengguna dan menurunkan tingkat bounce rate.

* Fitur "Lebih seperti ini": Di halaman detail film, pengguna akan melihat daftar film yang mirip secara tematis. Fitur ini dapat meminimalkan waktu yang dihabiskan pengguna untuk mencari konten dan mendorong mereka untuk terus menonton.

* Kampanye Pemasaran yang Ditargetkan: Model dapat digunakan untuk mengidentifikasi film-film yang mungkin disukai oleh pengguna yang tidak aktif. Rekomendasi yang dipersonalisasi dapat dikirim melalui email atau notifikasi untuk mendorong mereka kembali ke platform.

* Onboarding Pengguna Baru: Saat pengguna pertama kali mendaftar, mereka dapat diminta untuk memberi rating pada beberapa film. Sistem dapat menggunakan model Content-based untuk memberikan rekomendasi instan, menciptakan pengalaman yang menarik sejak awal.

## Kesimpulan

Secara keseluruhan, meskipun proyek ini berhasil membangun alur kerja rekomendasi, hasil evaluasi menunjukkan adanya kebutuhan mendesak untuk mengoptimalkan model, menyesuaikan bobot hibrida, dan mempertimbangkan strategi yang lebih efektif untuk mengatasi pengguna dengan riwayat rating yang minim.